# Getting started with Word2Vec in Gensim and making it work!

The idea behind Word2Vec is pretty simple. We are making and assumption that you can tell the meaning of a word by the company it keeps. This is analogous to the saying *show me your friends, and I'll tell who you are*. So if you have two words that have very similar neighbors (i.e. the usage context is about the same), then these words are probably quite similar in meaning or are at least highly related. For example, the words `shocked`,`appalled` and `astonished` are typically used in a similar context. 

In this tutorial, you will learn how to use the Gensim implementation of Word2Vec and actually get it to work! I have heard a lot of complaints about poor performance etc, but its really a combination of two things, (1) your input data and (2) your parameter settings. Note that the training algorithms in this package were ported from the [original Word2Vec implementation by Google](https://arxiv.org/pdf/1301.3781.pdf) and extended with additional functionality.

### Imports and logging

First, we start with our imports and get logging established:

In [5]:
# imports needed and set up logging
import gzip
import gensim 
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


### Dataset 
Next, is our dataset. The secret to getting Word2Vec really working for you is to have lots and lots of text data. In this case I am going to use data from the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset. This dataset has full user reviews of cars and hotels. I have specifically concatenated all of the hotel reviews into one big file which is about 97MB compressed and 229MB uncompressed. We will use the compressed file for this tutorial. Each line in this file represents a hotel review. You can download the OpinRank Word2Vec dataset here.

To avoid confusion, while gensim’s word2vec tutorial says that you need to pass it a sequence of sentences as its input, you can always pass it a whole review as a sentence (i.e. a much larger size of text), and it should not make much of a difference. 

Now, let's take a closer look at this data below by printing the first line. You can see that this is a pretty hefty review.

In [7]:
data_file="../corpus/reviews_data.txt.gz"

with gzip.open (data_file, 'rb') as f:
    for i,line in enumerate (f):
        print(line)
        break


b"Oct 12 2009 \tNice trendy hotel location not too bad.\tI stayed in this hotel for one night. As this is a fairly new place some of the taxi drivers did not know where it was and/or did not want to drive there. Once I have eventually arrived at the hotel, I was very pleasantly surprised with the decor of the lobby/ground floor area. It was very stylish and modern. I found the reception's staff geeting me with 'Aloha' a bit out of place, but I guess they are briefed to say that to keep up the coroporate image.As I have a Starwood Preferred Guest member, I was given a small gift upon-check in. It was only a couple of fridge magnets in a gift box, but nevertheless a nice gesture.My room was nice and roomy, there are tea and coffee facilities in each room and you get two complimentary bottles of water plus some toiletries by 'bliss'.The location is not great. It is at the last metro stop and you then need to take a taxi, but if you are not planning on going to see the historic sites in Be

### Read files into a list
Now that we've had a sneak peak of our dataset, we can read it into a list so that we can pass this on to the Word2Vec model. Notice in the code below, that I am directly reading the 
compressed file. I'm also doing a mild pre-processing of the reviews using `gensim.utils.simple_preprocess (line)`. This does some basic pre-processing such as tokenization, lowercasing, etc and returns back a list of tokens (words). Documentation of this pre-processing method can be found on the official [Gensim documentation site](https://radimrehurek.com/gensim/utils.html). 



In [8]:

def read_input(input_file):
    """This method reads the input file which is in gzip format"""
    
    logging.info("reading file {0}...this may take a while".format(input_file))
    
    with gzip.open (input_file, 'rb') as f:
        for i, line in enumerate (f): 

            if (i%10000==0):
                logging.info ("read {0} reviews".format (i))
            # do some pre-processing and return a list of words for each review text
            yield gensim.utils.simple_preprocess (line)

# read the tokenized reviews into a list
# each review item becomes a serries of words
# so this becomes a list of lists
documents = list (read_input (data_file))
logging.info ("Done reading data file")    

2020-06-16 15:14:05,829 : INFO : reading file ../corpus/reviews_data.txt.gz...this may take a while
2020-06-16 15:14:05,830 : INFO : read 0 reviews
2020-06-16 15:14:07,817 : INFO : read 10000 reviews
2020-06-16 15:14:09,694 : INFO : read 20000 reviews
2020-06-16 15:14:11,842 : INFO : read 30000 reviews
2020-06-16 15:14:13,865 : INFO : read 40000 reviews
2020-06-16 15:14:16,273 : INFO : read 50000 reviews
2020-06-16 15:14:18,415 : INFO : read 60000 reviews
2020-06-16 15:14:20,336 : INFO : read 70000 reviews
2020-06-16 15:14:22,033 : INFO : read 80000 reviews
2020-06-16 15:14:23,837 : INFO : read 90000 reviews
2020-06-16 15:14:25,513 : INFO : read 100000 reviews
2020-06-16 15:14:27,177 : INFO : read 110000 reviews
2020-06-16 15:14:28,918 : INFO : read 120000 reviews
2020-06-16 15:14:30,661 : INFO : read 130000 reviews
2020-06-16 15:14:32,520 : INFO : read 140000 reviews
2020-06-16 15:14:34,608 : INFO : read 150000 reviews
2020-06-16 15:14:37,770 : INFO : read 160000 reviews
2020-06-16 15

## Training the Word2Vec model

Training the model is fairly straightforward. You just instantiate Word2Vec and pass the reviews that we read in the previous step (the `documents`). So, we are essentially passing on a list of lists. Where each list within the main list contains a set of tokens from a user review. Word2Vec uses all these tokens to internally create a vocabulary. And by vocabulary, I mean a set of unique words.

After building the vocabulary, we just need to call `train(...)` to start training the Word2Vec model. Training on the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset takes about 10 minutes so please be patient while running your code on this dataset.

Behind the scenes we are actually training a simple neural network with a single hidden layer. But, we are actually not going to use the neural network after training. Instead, the goal is to learn the weights of the hidden layer. These weights are essentially the word vectors that we’re trying to learn. 

In [9]:
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)

2020-06-16 15:18:37,578 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2020-06-16 15:18:37,605 : INFO : collecting all words and their counts
2020-06-16 15:18:37,606 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-06-16 15:18:38,079 : INFO : PROGRESS: at sentence #10000, processed 1655714 words, keeping 25777 word types
2020-06-16 15:18:38,414 : INFO : PROGRESS: at sentence #20000, processed 3317863 words, keeping 35016 word types
2020-06-16 15:18:38,888 : INFO : PROGRESS: at sentence #30000, processed 5264072 words, keeping 47518 word types
2020-06-16 15:18:39,247 : INFO : PROGRESS: at sentence #40000, processed 7081746 words, keeping 56675 word types
2020-06-16 15:18:39,692 : INFO : PROGRESS: at sentence #50000, processed 9089491 words, keeping 63744 word types
2020-06-16 15:18:40,121 : INFO : PROGRESS: at sentence #60000, processed 11013726 words, keeping 76786 word types
2020-06-16 15:18:40,484 : INFO : PROGRESS: a

2020-06-16 15:19:29,009 : INFO : EPOCH 2 - PROGRESS: at 63.11% examples, 1754616 words/s, in_qsize 17, out_qsize 2
2020-06-16 15:19:30,012 : INFO : EPOCH 2 - PROGRESS: at 68.24% examples, 1730987 words/s, in_qsize 17, out_qsize 2
2020-06-16 15:19:31,018 : INFO : EPOCH 2 - PROGRESS: at 74.04% examples, 1729330 words/s, in_qsize 19, out_qsize 0
2020-06-16 15:19:32,022 : INFO : EPOCH 2 - PROGRESS: at 79.47% examples, 1727771 words/s, in_qsize 19, out_qsize 0
2020-06-16 15:19:33,031 : INFO : EPOCH 2 - PROGRESS: at 85.06% examples, 1726609 words/s, in_qsize 18, out_qsize 1
2020-06-16 15:19:34,037 : INFO : EPOCH 2 - PROGRESS: at 90.49% examples, 1713808 words/s, in_qsize 18, out_qsize 1
2020-06-16 15:19:35,037 : INFO : EPOCH 2 - PROGRESS: at 96.09% examples, 1709309 words/s, in_qsize 17, out_qsize 2
2020-06-16 15:19:35,717 : INFO : worker thread finished; awaiting finish of 9 more threads
2020-06-16 15:19:35,729 : INFO : worker thread finished; awaiting finish of 8 more threads
2020-06-16 15

2020-06-16 15:20:15,356 : INFO : EPOCH 5 - PROGRESS: at 5.07% examples, 1566432 words/s, in_qsize 18, out_qsize 1
2020-06-16 15:20:16,357 : INFO : EPOCH 5 - PROGRESS: at 10.01% examples, 1594482 words/s, in_qsize 18, out_qsize 1
2020-06-16 15:20:17,363 : INFO : EPOCH 5 - PROGRESS: at 14.44% examples, 1587019 words/s, in_qsize 18, out_qsize 1
2020-06-16 15:20:18,368 : INFO : EPOCH 5 - PROGRESS: at 18.90% examples, 1588179 words/s, in_qsize 19, out_qsize 0
2020-06-16 15:20:19,373 : INFO : EPOCH 5 - PROGRESS: at 23.29% examples, 1588641 words/s, in_qsize 19, out_qsize 0
2020-06-16 15:20:20,373 : INFO : EPOCH 5 - PROGRESS: at 28.63% examples, 1590683 words/s, in_qsize 17, out_qsize 2
2020-06-16 15:20:21,375 : INFO : EPOCH 5 - PROGRESS: at 34.07% examples, 1583762 words/s, in_qsize 19, out_qsize 0
2020-06-16 15:20:22,382 : INFO : EPOCH 5 - PROGRESS: at 39.55% examples, 1578945 words/s, in_qsize 18, out_qsize 1
2020-06-16 15:20:23,386 : INFO : EPOCH 5 - PROGRESS: at 45.39% examples, 1578929 

2020-06-16 15:21:05,828 : INFO : EPOCH 2 - PROGRESS: at 67.54% examples, 1714806 words/s, in_qsize 19, out_qsize 0
2020-06-16 15:21:06,839 : INFO : EPOCH 2 - PROGRESS: at 73.17% examples, 1711288 words/s, in_qsize 18, out_qsize 1
2020-06-16 15:21:07,839 : INFO : EPOCH 2 - PROGRESS: at 78.71% examples, 1711892 words/s, in_qsize 19, out_qsize 0
2020-06-16 15:21:08,843 : INFO : EPOCH 2 - PROGRESS: at 84.41% examples, 1713534 words/s, in_qsize 16, out_qsize 3
2020-06-16 15:21:09,844 : INFO : EPOCH 2 - PROGRESS: at 90.50% examples, 1715378 words/s, in_qsize 19, out_qsize 0
2020-06-16 15:21:10,846 : INFO : EPOCH 2 - PROGRESS: at 96.41% examples, 1715777 words/s, in_qsize 16, out_qsize 3
2020-06-16 15:21:11,398 : INFO : worker thread finished; awaiting finish of 9 more threads
2020-06-16 15:21:11,404 : INFO : worker thread finished; awaiting finish of 8 more threads
2020-06-16 15:21:11,405 : INFO : worker thread finished; awaiting finish of 7 more threads
2020-06-16 15:21:11,410 : INFO : work

2020-06-16 15:21:51,700 : INFO : EPOCH 5 - PROGRESS: at 25.24% examples, 1722616 words/s, in_qsize 19, out_qsize 0
2020-06-16 15:21:52,713 : INFO : EPOCH 5 - PROGRESS: at 31.68% examples, 1724201 words/s, in_qsize 18, out_qsize 1
2020-06-16 15:21:53,714 : INFO : EPOCH 5 - PROGRESS: at 37.76% examples, 1729950 words/s, in_qsize 18, out_qsize 1
2020-06-16 15:21:54,718 : INFO : EPOCH 5 - PROGRESS: at 44.14% examples, 1732878 words/s, in_qsize 19, out_qsize 0
2020-06-16 15:21:55,721 : INFO : EPOCH 5 - PROGRESS: at 50.30% examples, 1734189 words/s, in_qsize 18, out_qsize 1
2020-06-16 15:21:56,723 : INFO : EPOCH 5 - PROGRESS: at 56.15% examples, 1733262 words/s, in_qsize 16, out_qsize 3
2020-06-16 15:21:57,729 : INFO : EPOCH 5 - PROGRESS: at 61.89% examples, 1726554 words/s, in_qsize 18, out_qsize 1
2020-06-16 15:21:58,733 : INFO : EPOCH 5 - PROGRESS: at 68.07% examples, 1729718 words/s, in_qsize 19, out_qsize 0
2020-06-16 15:21:59,744 : INFO : EPOCH 5 - PROGRESS: at 74.02% examples, 1730972

2020-06-16 15:22:41,482 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-06-16 15:22:41,484 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-06-16 15:22:41,485 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-06-16 15:22:41,490 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-06-16 15:22:41,491 : INFO : EPOCH - 7 : training on 41519358 raw words (30345535 effective words) took 18.8s, 1613528 effective words/s
2020-06-16 15:22:42,502 : INFO : EPOCH 8 - PROGRESS: at 4.01% examples, 1236079 words/s, in_qsize 18, out_qsize 1
2020-06-16 15:22:43,510 : INFO : EPOCH 8 - PROGRESS: at 8.65% examples, 1328686 words/s, in_qsize 16, out_qsize 3
2020-06-16 15:22:44,517 : INFO : EPOCH 8 - PROGRESS: at 13.01% examples, 1421618 words/s, in_qsize 18, out_qsize 1
2020-06-16 15:22:45,519 : INFO : EPOCH 8 - PROGRESS: at 17.69% examples, 1477634 words/s, in_qsize 17, out_qsize 2
2020-06-16 15:22:46,529 : INFO : EPOC

2020-06-16 15:23:33,247 : INFO : EPOCH 10 - PROGRESS: at 54.10% examples, 1393789 words/s, in_qsize 16, out_qsize 3
2020-06-16 15:23:34,248 : INFO : EPOCH 10 - PROGRESS: at 59.44% examples, 1403712 words/s, in_qsize 18, out_qsize 1
2020-06-16 15:23:35,254 : INFO : EPOCH 10 - PROGRESS: at 65.09% examples, 1415927 words/s, in_qsize 18, out_qsize 1
2020-06-16 15:23:36,262 : INFO : EPOCH 10 - PROGRESS: at 70.30% examples, 1427349 words/s, in_qsize 17, out_qsize 2
2020-06-16 15:23:37,263 : INFO : EPOCH 10 - PROGRESS: at 76.18% examples, 1447900 words/s, in_qsize 20, out_qsize 0
2020-06-16 15:23:38,264 : INFO : EPOCH 10 - PROGRESS: at 81.65% examples, 1462831 words/s, in_qsize 18, out_qsize 1
2020-06-16 15:23:39,266 : INFO : EPOCH 10 - PROGRESS: at 87.54% examples, 1478146 words/s, in_qsize 18, out_qsize 1
2020-06-16 15:23:40,268 : INFO : EPOCH 10 - PROGRESS: at 93.62% examples, 1492427 words/s, in_qsize 19, out_qsize 0
2020-06-16 15:23:41,269 : INFO : EPOCH 10 - PROGRESS: at 99.62% examples

(303480878, 415193580)

## Now, let's look at some output 
This first example shows a simple case of looking up words similar to the word `dirty`. All we need to do here is to call the `most_similar` function and provide the word `dirty` as the positive example. This returns the top 10 similar words. 

In [ ]:

w1 = "dirty"
model.wv.most_similar (positive=w1)


That looks pretty good, right? Let's look at a few more. Let's look at similarity for `polite`, `france` and `shocked`. 

In [ ]:
# look up top 6 words similar to 'polite'
w1 = ["polite"]
model.wv.most_similar (positive=w1,topn=6)


In [ ]:
# look up top 6 words similar to 'france'
w1 = ["france"]
model.wv.most_similar (positive=w1,topn=6)


In [ ]:
# look up top 6 words similar to 'shocked'
w1 = ["shocked"]
model.wv.most_similar (positive=w1,topn=6)


That's, nice. You can even specify several positive examples to get things that are related in the provided context and provide negative examples to say what should not be considered as related. In the example below we are asking for all items that *relate to bed* only:

In [ ]:
# get everything related to stuff on the bed
w1 = ["bed",'sheet','pillow']
w2 = ['couch']
model.wv.most_similar (positive=w1,negative=w2,topn=10)


### Similarity between two words in the vocabulary

You can even use the Word2Vec model to return the similarity between two words that are present in the vocabulary. 

In [ ]:
# similarity between two different words
model.wv.similarity(w1="dirty",w2="smelly")

In [ ]:
# similarity between two identical words
model.wv.similarity(w1="dirty",w2="dirty")

In [ ]:
# similarity between two unrelated words
model.wv.similarity(w1="dirty",w2="clean")

Under the hood, the above three snippets computes the cosine similarity between the two specified words using word vectors of each. From the scores, it makes sense that `dirty` is highly similar to `smelly` but `dirty` is dissimilar to `clean`. If you do a similarity between two identical words, the score will be 1.0 as the range of the cosine similarity score will always be between [0.0-1.0]. You can read more about cosine similarity scoring [here](https://en.wikipedia.org/wiki/Cosine_similarity).

### Find the odd one out
You can even use Word2Vec to find odd items given a list of items.

In [ ]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["cat","dog","france"])

In [ ]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["bed","pillow","duvet","shower"])


## Understanding some of the parameters
To train the model earlier, we had to set some parameters. Now, let's try to understand what some of them mean. For reference, this is the command that we used to train the model.

```
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
```

### `size`
The size of the dense vector to represent each token or word. If you have very limited data, then size should be a much smaller value. If you have lots of data, its good to experiment with various sizes. A value of 100-150 has worked well for me. 

### `window`
The maximum distance between the target word and its neighboring word. If your neighbor's position is greater than the maximum window width to the left and the right, then, some neighbors are not considered as being related to the target word. In theory, a smaller window should give you terms that are more related. If you have lots of data, then the window size should not matter too much, as long as its a decent sized window. 

### `min_count`
Minimium frequency count of words. The model would ignore words that do not statisfy the `min_count`. Extremely infrequent words are usually unimportant, so its best to get rid of those. Unless your dataset is really tiny, this does not really affect the model.

### `workers`
How many threads to use behind the scenes?


## When should you use Word2Vec?

There are many application scenarios for Word2Vec. Imagine if you need to build a sentiment lexicon. Training a Word2Vec model on large amounts of user reviews helps you achieve that. You have a lexicon for not just sentiment, but for most words in the vocabulary. 

Beyond, raw unstructured text data, you could also use Word2Vec for more structured data. For example, if you had tags for a million stackoverflow questions and answers, you could find tags that are related to a given tag and recommend the related ones for exploration. You can do this by treating each set of co-occuring tags as a "sentence" and train a Word2Vec model on this data. Granted, you still need a large number of examples to make it work. 


# Storing and loading models

In [20]:
# you can store/load models using the standard gensim methods

from gensim.models import Word2Vec

model.save("word2vec.model")

new_model = Word2Vec.load("word2vec.model")

# 加载模型后再训练
new_model.train([["hello", "world"]], total_examples=1, epochs=1)

new_model.wv.doesnt_match(["cat","dog","france"])
new_model['hello']

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


array([ 3.98078227e+00, -9.06566024e-01, -1.34488329e-01, -5.06701756e+00,
        1.52403259e+00, -3.95868564e+00,  2.04536891e+00, -3.71955180e+00,
        1.74870801e+00,  1.72618759e+00, -5.17557716e+00, -3.03478628e-01,
        2.88443565e+00,  4.68250418e+00,  3.72485495e+00, -7.50020885e+00,
        1.38099241e+00, -5.20219803e-01,  5.00840187e-01,  1.85577202e+00,
        3.22760773e+00, -2.51910901e+00,  7.05691874e-01,  2.68260121e-01,
        7.77740300e-01, -2.86655974e+00, -3.14675546e+00, -5.54975843e+00,
        3.14233947e+00,  3.96534038e+00, -3.96096754e+00,  1.98728049e+00,
        2.33296466e+00,  3.40679944e-01, -2.71325618e-01,  8.66360486e-01,
       -8.76150072e-01,  2.02023840e+00, -1.92977881e+00, -3.56002331e+00,
        8.79617453e-01,  3.02764028e-01,  2.13232636e+00,  3.07156473e-01,
       -2.11493230e+00,  5.12150349e-03, -2.14534473e+00, -9.21717465e-01,
        3.67876244e+00,  3.16294146e+00,  2.78425837e+00,  2.01577902e+00,
        4.85608354e-02, -

# 训练中文的例子

## 分词
首先使用jieba分词工具对文本进行分词处理，处理完成放置到新的文件

In [ ]:
import jieba
import jieba.analyse

# suggest_freq 设置词为整体，不被分离

jieba.suggest_freq('沙瑞金', True)
jieba.suggest_freq('田国富', True)
jieba.suggest_freq('高育良', True)
jieba.suggest_freq('侯亮平', True)
jieba.suggest_freq('钟小艾', True)
jieba.suggest_freq('陈岩石', True)
jieba.suggest_freq('欧阳菁', True)
jieba.suggest_freq('易学习', True)
jieba.suggest_freq('王大路', True)
jieba.suggest_freq('蔡成功', True)
jieba.suggest_freq('孙连城', True)
jieba.suggest_freq('季昌明', True)
jieba.suggest_freq('丁义珍', True)
jieba.suggest_freq('郑西坡', True)
jieba.suggest_freq('赵东来', True)
jieba.suggest_freq('高小琴', True)
jieba.suggest_freq('赵瑞龙', True)
jieba.suggest_freq('林华华', True)
jieba.suggest_freq('陆亦可', True)
jieba.suggest_freq('刘新建', True)
jieba.suggest_freq('刘庆祝', True)

with open('../corpus/in_the_name_of_people.txt', 'rb') as f:
    document = f.read()
#     document_decode = document.decode('GBK')
    document_cut = jieba.cut(document)
    result = ' '.join(document_cut)
    result = result.encode('utf-8')
    with open('../corpus/in_the_name_of_people_segment.txt', 'wb') as f2:
        f2.write(result)

f.close()
f2.close()   


## 模型建立

In [ ]:
from gensim.models import word2vec


min_word_count = 10 # ⼀个word，最少出现多少次 才被计⼊
num_workers = 1 # 多少thread⼀起跑（快⼀点⼉）
size = 300 # vec的size
window = 3 # 前后观察多⻓的“语境”


sentences = word2vec.LineSentence('../corpus/in_the_name_of_people_segment.txt') 


model = word2vec.Word2Vec(sentences, size=size, workers=num_workers, \
                          min_count = min_word_count, \
                          window = window)

model.train(sentences, total_examples=sentences.max_sentence_length, epochs=10)

## 模型词典库

In [ ]:
for key, value in model.wv.vocab.items():
    print(key, value)

## 相似度计算

In [ ]:
model.wv.most_similar(positive='侯亮平')

In [ ]:
model.wv.similarity('沙发', '我')

## 与侯亮平最相似的前10个，只显示三个字的词

In [ ]:
for value in model.wv.similar_by_word('侯亮平', topn=10):
    if len(value[0]) == 3:
        print(value)

## 词向量

In [ ]:
model.wv.__getitem__('人民')